In [1]:
sc.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 
sc.install_pypi_package("pip==21.3.1")
sc.install_pypi_package("matplotlib", "https://pypi.org/simple")
# sc.install_pypi_package("seaborn")

import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql import functions as F
from pyspark.sql.functions import when, lit
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType


from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline 
from pyspark.mllib.evaluation import MulticlassMetrics


import pandas
# import seaborn as sns
#import matplotlib.pyplot as plt

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
15,application_1637155679397_0016,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/7e/ab/ea76361f9d3e732e114adcd801d2820d5319c23d0ac5482fa3b412db217e/pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/a4/6d/6463d49a933f547439d6b5b98b46af8742cc03ae83543e4d7688c2420f8b/pip-21.3.1-py3-none-any.whl
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1

  Using cached matplotlib-3.5.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached numpy-1.21.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.28.3-py3-none-any.whl (884 kB)
  Using cached kiwisolver-1.3.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.1 MB)
  Us

In [2]:
spark = SparkSession.builder.master("local").appName("EMR_Test").config('spark.ui.port', '4050').getOrCreate()
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df = spark.read.csv("s3://emr-pipeline-data-folder/Telco-Customer-Churn.csv", header=True, inferSchema=True)
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

In [4]:
df.show(5)  #shows first five rows of dataframe
df.count()  #displays the number of rows present in the dataframe

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|    

In [5]:
#df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.printSchema() #describes the schema of the dataframe

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)

In [7]:
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show() #checks for the null values

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|         0|     0|            0|      0|         0|     0|           0|            0|              0|             0|           0|               0|          0|          0|              0|       0|               0| 

In [8]:
df.describe().show()  #basic statistical summary of all numeric attributes in dataframe

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|summary|customerID|gender|     SeniorCitizen|Partner|Dependents|            tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|    MonthlyCharges|      TotalCharges|Churn|
+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|  count|      7043|  7043|              7043|   7043|      7043|     

In [9]:
df1 = df.drop("customerID")  #As CustomerId values are unique it can't be converted to a integer value to fit into the model 
df1.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)

In [10]:
df1.groupBy('Churn').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+
|Churn|count|
+-----+-----+
|  Yes| 1869|
|   No| 5174|
+-----+-----+

In [11]:
df1.select("TotalCharges").distinct().show()  #shows unique values in a column

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|TotalCharges|
+------------+
|     1874.45|
|        74.5|
|     5586.45|
|       667.7|
|     5060.85|
|      258.35|
|      2983.8|
|      5432.2|
|     1776.45|
|      453.75|
|       518.3|
|      2722.2|
|       189.2|
|     1425.45|
|        78.9|
|       338.9|
|      1784.5|
|      6479.4|
|      1468.9|
|      1058.6|
+------------+
only showing top 20 rows

In [12]:
#To replace each value in a column to respective integer value
df1 = df1.withColumn('Partner', when(df1.Partner=='Yes', 
lit(1)).otherwise(0))
df1 = df1.withColumn('DeviceProtection', when(df1.DeviceProtection=='Yes', 
lit(1)).when(df1.DeviceProtection=='No internet Service',lit(-1)).otherwise(0))
df1 = df1.withColumn('gender', when(df1.gender=='Female', 
lit(1)).otherwise(0))
df1 = df1.withColumn('Dependents', when(df1.Dependents=='Yes', 
lit(1)).otherwise(0))
df1 = df1.withColumn('PhoneService', when(df1.PhoneService=='Yes', 
lit(1)).otherwise(0))
df1 = df1.withColumn('MultipleLines', when(df1.MultipleLines=='Yes', 
lit(1)).when(df1.DeviceProtection=='No phone service',lit(-1)).otherwise(0))
df1 = df1.withColumn('InternetService', when(df1.InternetService=='Fiber optic', 
lit(1)).when(df1.InternetService=='DSL',lit(2)).otherwise(0))
df1 = df1.withColumn('OnlineSecurity', when(df1.OnlineSecurity=='Yes', 
lit(1)).when(df1.OnlineSecurity=='No internet Service',lit(-1)).otherwise(0))
df1 = df1.withColumn('OnlineBackup', when(df1.OnlineBackup=='Yes', 
lit(1)).when(df1.OnlineBackup=='No internet Service',lit(-1)).otherwise(0))
df1 = df1.withColumn('TechSupport', when(df1.TechSupport=='Yes', 
lit(1)).when(df1.TechSupport=='No internet Service',lit(-1)).otherwise(0))
df1 = df1.withColumn('Churn', when(df1.Churn=='Yes', 
lit(1)).otherwise(0))
df1 = df1.withColumn('StreamingTV', when(df1.StreamingTV=='Yes', 
lit(1)).when(df1.StreamingTV=='No internet Service',lit(-1)).otherwise(0))
df1 = df1.withColumn('StreamingMovies', when(df1.StreamingMovies=='Yes', 
lit(1)).when(df1.StreamingMovies=='No internet Service',lit(-1)).otherwise(0))
df1 = df1.withColumn('PaperlessBilling', when(df1.PaperlessBilling=='Yes', 
lit(1)).otherwise(0))
df1 = df1.withColumn('Contract', when(df1.Contract=='Month-to-month', 
lit(0)).when(df1.Contract=='One year',lit(1)).otherwise(2)) 
df1 = df1.withColumn('PaymentMethod', when(df1.PaymentMethod=='Electronic check', 
lit(0)).when(df1.PaymentMethod=='Mailed check',lit(1)).when(df1.PaymentMethod=='Bank transfer(automatic)',lit(2)).otherwise(3)) 


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# To replace null values with mean value of the attribute

df_stats = df1.select(
    _mean(col('TotalCharges')).alias('mean')
).collect()

mean = df_stats[0]['mean']
df1 = df1.na.fill(value=mean)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df1 = df1.withColumn("TotalCharges", df1["TotalCharges"].cast(IntegerType()))   #Converting String to integer

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df1.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df1.columns]).show()  #to check for the null values

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|     0|            0|      0|         0|     0|           0|            0|              0|             0|           0|               0|          0|          0|              0|       0|               0|            0|             0|           0|   

In [16]:
df1.describe().show() #statistical summary of all numeric attribute

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+
|summary|            gender|     SeniorCitizen|            Partner|        Dependents|            tenure|       PhoneService|      MultipleLines|   InternetService|    OnlineSecurity|      OnlineBackup|   DeviceProtection|       TechSupport|        StreamingTV|   StreamingMovies|          Contract|  PaperlessBilling|     PaymentMethod|    MonthlyCharges|      TotalCharges|              Churn|
+-------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+------

In [17]:
x = df1.drop('customerID','PhoneService','OnlineBackup','TechSupport','DeviceProtection','StreamingMovies','PaymentMethod','Dependents')     

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
x.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- gender: integer (nullable = false)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: integer (nullable = false)
 |-- tenure: integer (nullable = true)
 |-- MultipleLines: integer (nullable = false)
 |-- InternetService: integer (nullable = false)
 |-- OnlineSecurity: integer (nullable = false)
 |-- StreamingTV: integer (nullable = false)
 |-- Contract: integer (nullable = false)
 |-- PaperlessBilling: integer (nullable = false)
 |-- MonthlyCharges: double (nullable = false)
 |-- TotalCharges: integer (nullable = true)
 |-- Churn: integer (nullable = false)

In [19]:
x.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in x.columns]).show()  #to check for the null values

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------+-------+------+-------------+---------------+--------------+-----------+--------+----------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|tenure|MultipleLines|InternetService|OnlineSecurity|StreamingTV|Contract|PaperlessBilling|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+------+-------------+---------------+--------------+-----------+--------+----------------+--------------+------------+-----+
|     0|            0|      0|     0|            0|              0|             0|          0|       0|               0|             0|           0|    0|
+------+-------------+-------+------+-------------+---------------+--------------+-----------+--------+----------------+--------------+------------+-----+

In [20]:
# used minmaxscaler to transform each feature into a specific range
feature_names = ['gender',
 'SeniorCitizen',
 'Partner',
 'tenure',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'StreamingTV',
 'Contract',
 'PaperlessBilling',
 'MonthlyCharges',
 'TotalCharges']
assembler = VectorAssembler(inputCols=feature_names, outputCol="features")
scalar = MinMaxScaler(inputCol="features ", outputCol="Churn")
x = assembler.transform(x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
train, test = x.randomSplit([0.8, 0.2], seed = 2018) # splitting data as 80% into train and 20% into test

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# build, train and evaluate a model using logisticregression from pyspar.ml library
lr = LogisticRegression(maxIter=100, labelCol='Churn')
model = lr.fit(train)
pipeline = Pipeline(stages = [lr])
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [.1, .01]).build() ## we are doing a grid search on different parameters
val = CrossValidator(estimator=pipeline, estimatorParamMaps=BinaryClassificationEvaluator(), numFolds = 2) ## performing a 2-fold cross validator
prediction = model.transform(test)
evaluator = BinaryClassificationEvaluator(labelCol='Churn')
print(evaluator.evaluate(prediction))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8379610459320607

In [23]:

#accuracy = prediction.filter(prediction.Churn == prediction.prediction).count() / float(prediction.count())
#print(accuracy)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
prediction.select('Churn','prediction', 'probability').show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------+--------------------+
|Churn|prediction|         probability|
+-----+----------+--------------------+
|    0|       0.0|[0.75959098630878...|
|    1|       0.0|[0.75887166702708...|
|    0|       0.0|[0.75794103335116...|
|    0|       0.0|[0.75721835520043...|
|    0|       0.0|[0.75721835520043...|
|    1|       0.0|[0.75721835520043...|
|    1|       0.0|[0.75703745680286...|
|    0|       0.0|[0.75631294831562...|
|    1|       0.0|[0.75613159256134...|
|    0|       0.0|[0.75595014539116...|
|    0|       0.0|[0.75576860682380...|
|    1|       0.0|[0.75576860682380...|
|    0|       0.0|[0.67562166627040...|
|    0|       0.0|[0.67454275004340...|
|    1|       0.0|[0.67299386148310...|
|    1|       1.0|[0.41333964921484...|
|    1|       1.0|[0.41190907772043...|
|    1|       1.0|[0.40901365193531...|
|    1|       1.0|[0.38574006771983...|
|    1|       1.0|[0.33625673330361...|
+-----+----------+--------------------+
only showing top 20 rows